In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set()
from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import sklearn

In [33]:
#train = pd.read_csv('./data_1-First/Fina_Train.csv')
#test = pd.read_csv('./data_1-First/Fina_Test.csv')
train = pd.read_csv('./Data/Train-All-Selected-Cities.csv')
test = pd.read_csv('./Data/Test-All-Selected-Cities.csv')

In [34]:
y = train['TARGET(PRICE_IN_LACS)']
x = train.drop(['TARGET(PRICE_IN_LACS)'],axis=1)


In [35]:
x.columns

Index(['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK',
       'SQUARE_FT', 'RESALE', 'LONGITUDE', 'LATITUDE', 'Rare_House',
       'City_Type', 'CITY', 'CITY_IN'],
      dtype='object')

In [36]:
cat_col = ['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK','RESALE', 'CITY', 'CITY_IN','Rare_House', 'City_Type']
num_col = list(set(x.columns.values)-set(cat_col))

In [37]:
num_col

['SQUARE_FT', 'LONGITUDE', 'LATITUDE']

In [39]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [40]:
encoder = OneHotEncoder(drop='first')
x_encoded = encoder.fit_transform(x[cat_col])

In [41]:
encoded_df=pd.DataFrame(columns=encoder.get_feature_names(cat_col),data=x_encoded.toarray())

In [42]:
new_x = x.merge(encoded_df,on=x.index)

In [43]:
del encoded_df

In [44]:
del x

In [45]:
new_x = new_x.drop(cat_col,axis=1)
new_x = new_x.iloc[:,1:]

In [47]:
new_x.columns

Index(['SQUARE_FT', 'LONGITUDE', 'LATITUDE', 'POSTED_BY_1', 'POSTED_BY_2',
       'UNDER_CONSTRUCTION_1', 'RERA_1', 'BHK_NO._2', 'BHK_NO._3', 'BHK_NO._4',
       'BHK_NO._5', 'BHK_OR_RK_1', 'RESALE_1', 'CITY_5', 'CITY_6', 'CITY_7',
       'CITY_8', 'CITY_9', 'CITY_10', 'CITY_11', 'CITY_15', 'CITY_16',
       'CITY_17', 'CITY_18', 'CITY_19', 'CITY_23', 'CITY_24', 'CITY_28',
       'CITY_29', 'CITY_31', 'CITY_32', 'CITY_34', 'CITY_35', 'CITY_37',
       'CITY_38', 'CITY_40', 'CITY_41', 'CITY_42', 'CITY_43', 'CITY_46',
       'CITY_50', 'CITY_51', 'CITY_53', 'CITY_54', 'CITY_56', 'CITY_59',
       'CITY_IN_14', 'CITY_IN_15', 'CITY_IN_21', 'CITY_IN_29', 'CITY_IN_31',
       'CITY_IN_35', 'CITY_IN_36', 'CITY_IN_39', 'CITY_IN_40', 'CITY_IN_54',
       'CITY_IN_59', 'CITY_IN_60', 'CITY_IN_67', 'CITY_IN_70', 'CITY_IN_80',
       'CITY_IN_96', 'CITY_IN_97', 'CITY_IN_109', 'CITY_IN_112',
       'Rare_House_1', 'City_Type_1', 'City_Type_2', 'City_Type_3'],
      dtype='object')

In [48]:
scaler = StandardScaler()
#scaled_x = scaler.fit_transform(new_x[num_col])
scaled_x = scaler.fit_transform(new_x[num_col])

In [49]:
new_x.loc[:,num_col] =scaled_x

In [50]:
del scaled_x

In [35]:
from sklearn.decomposition import PCA

In [37]:
pca = PCA(0.95)
low_x = pca.fit_transform(new_x)

In [ ]:
np.sqrt(mean_squared_log_error(y_valid,model.predict(x_valid)))

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [54]:
x_train,x_test,y_train,y_test = train_test_split(new_x,y,test_size=0.2,random_state=37)

In [55]:
x_train.shape,x_test.shape

((20859, 69), (5215, 69))

In [56]:
from sklearn.metrics import mean_squared_log_error

In [57]:
from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS

In [58]:
reg = OLS(y_train,x_train)
reg = reg.fit()
print(reg.summary())
train_acc = 100-mean_squared_log_error(np.exp(y_train),np.exp(reg.predict(x_train)))*100
test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(reg.predict(x_test)))*100
#algo.append('OLS')
#train_accuracy.append(train_acc)
#test_accuracy.append(test_acc)
print('Train\t|\t',train_acc)
print('Test\t|\t',test_acc)
print('RMSLE\t|\t',np.sqrt(mean_squared_log_error(y_test,reg.predict(x_test))))


                                  OLS Regression Results                                  
Dep. Variable:     TARGET(PRICE_IN_LACS)   R-squared (uncentered):                   0.992
Model:                               OLS   Adj. R-squared (uncentered):              0.992
Method:                    Least Squares   F-statistic:                          3.872e+04
Date:                   Thu, 22 Oct 2020   Prob (F-statistic):                        0.00
Time:                           15:33:59   Log-Likelihood:                         -9210.0
No. Observations:                  20859   AIC:                                  1.855e+04
Df Residuals:                      20794   BIC:                                  1.907e+04
Df Model:                             65                                                  
Covariance Type:               nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-

In [59]:
reg = LinearRegression(normalize=True)
reg = reg.fit(x_train,y_train)
pred = reg.predict(x_train)
train_acc = 100-mean_squared_log_error(np.exp(y_train),np.exp(reg.predict(x_train)))*100
test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(reg.predict(x_test)))*100
#algo.append('OLS')
#train_accuracy.append(train_acc)
#test_accuracy.append(test_acc)
print('Train\t|\t',train_acc)
print('Test\t|\t',test_acc)
print('RMSLE\t|\t',np.sqrt(mean_squared_log_error(y_test,reg.predict(x_test))))


Train	|	 87.28291612887277
Test	|	 88.0564439029206
RMSLE	|	 0.06988865474436949


In [60]:
from sklearn.svm import SVR

In [61]:
reg = SVR(verbose=True)
score = reg.fit(x_train,y_train)
pred = score.predict(x_train)
train_acc = 100-mean_squared_log_error(np.exp(y_train),np.exp(pred))*100
test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(score.predict(x_test)))*100

print('Train\t|\t',train_acc)
print('Test\t|\t',test_acc)
print('RMSLE\t|\t',np.sqrt(mean_squared_log_error(y_test,reg.predict(x_test))))



[LibSVM]Train	|	 89.83397011193313
Test	|	 89.93088526934726
RMSLE	|	 0.06388668764065836


In [62]:
from xgboost import XGBRegressor,plot_importance,plot_tree
import xgboost as xgb

In [63]:
x_train,x_test,y_train,y_test = train_test_split(new_x,y,train_size=0.7,random_state=37)

x_test,x_valid,y_test,y_valid = train_test_split(x_test,y_test,train_size=0.5,random_state=2)
x_train.shape,x_test.shape,x_valid.shape

((18251, 69), (3911, 69), (3912, 69))

In [64]:
#reg = XGBRegressor(objective='reg:squaredlogerror' ,n_jobs=-1,n_estimators=500,verbosity=1,eta=0.4,gamma=0.001,max_depth= 6, min_child_weight= 1)
eval_set = [(x_train, y_train),(x_test, y_test),(x_valid,y_valid) ]
reg = XGBRegressor(objective='reg:squaredlogerror' ,n_jobs=4,n_estimators=3000,gamma=1e-05,eta=0.1,max_depth=4, min_child_weight=0.001)

score = reg.fit(x_train,y_train,verbose=True,eval_metric=['rmsle'],eval_set=eval_set,early_stopping_rounds=20)
pred = score.predict(x_train)
train_acc = 100-mean_squared_log_error(np.exp(y_train),np.exp(pred))*100
test_acc = 100-mean_squared_log_error(np.exp(y_test),np.exp(score.predict(x_test)))*100
#train_acc = 100-mean_squared_log_error(y_train,pred)*100
#test_acc = 100-mean_squared_log_error(y_test,score.predict(x_test))*100

print('Train',train_acc)
print('Test',test_acc)


[0]	validation_0-rmsle:1.16825	validation_1-rmsle:1.17213	validation_2-rmsle:1.16567
Multiple eval metrics have been passed: 'validation_2-rmsle' will be used for early stopping.

Will train until validation_2-rmsle hasn't improved in 20 rounds.
[1]	validation_0-rmsle:1.11627	validation_1-rmsle:1.12007	validation_2-rmsle:1.11371
[2]	validation_0-rmsle:1.06539	validation_1-rmsle:1.06910	validation_2-rmsle:1.06286
[3]	validation_0-rmsle:1.01565	validation_1-rmsle:1.01927	validation_2-rmsle:1.01314
[4]	validation_0-rmsle:0.96708	validation_1-rmsle:0.97061	validation_2-rmsle:0.96460
[5]	validation_0-rmsle:0.91971	validation_1-rmsle:0.92315	validation_2-rmsle:0.91726
[6]	validation_0-rmsle:0.87358	validation_1-rmsle:0.87693	validation_2-rmsle:0.87116
[7]	validation_0-rmsle:0.82871	validation_1-rmsle:0.83196	validation_2-rmsle:0.82632
[8]	validation_0-rmsle:0.78514	validation_1-rmsle:0.78829	validation_2-rmsle:0.78278
[9]	validation_0-rmsle:0.74290	validation_1-rmsle:0.74595	validation_2-rms

[94]	validation_0-rmsle:0.06683	validation_1-rmsle:0.06708	validation_2-rmsle:0.06742
[95]	validation_0-rmsle:0.06678	validation_1-rmsle:0.06703	validation_2-rmsle:0.06736
[96]	validation_0-rmsle:0.06674	validation_1-rmsle:0.06699	validation_2-rmsle:0.06731
[97]	validation_0-rmsle:0.06668	validation_1-rmsle:0.06692	validation_2-rmsle:0.06724
[98]	validation_0-rmsle:0.06663	validation_1-rmsle:0.06686	validation_2-rmsle:0.06719
[99]	validation_0-rmsle:0.06658	validation_1-rmsle:0.06682	validation_2-rmsle:0.06715
[100]	validation_0-rmsle:0.06651	validation_1-rmsle:0.06676	validation_2-rmsle:0.06711
[101]	validation_0-rmsle:0.06619	validation_1-rmsle:0.06647	validation_2-rmsle:0.06679
[102]	validation_0-rmsle:0.06615	validation_1-rmsle:0.06643	validation_2-rmsle:0.06676
[103]	validation_0-rmsle:0.06611	validation_1-rmsle:0.06638	validation_2-rmsle:0.06671
[104]	validation_0-rmsle:0.06602	validation_1-rmsle:0.06631	validation_2-rmsle:0.06662
[105]	validation_0-rmsle:0.06598	validation_1-rms

[189]	validation_0-rmsle:0.06166	validation_1-rmsle:0.06246	validation_2-rmsle:0.06262
[190]	validation_0-rmsle:0.06160	validation_1-rmsle:0.06241	validation_2-rmsle:0.06255
[191]	validation_0-rmsle:0.06158	validation_1-rmsle:0.06239	validation_2-rmsle:0.06254
[192]	validation_0-rmsle:0.06154	validation_1-rmsle:0.06236	validation_2-rmsle:0.06252
[193]	validation_0-rmsle:0.06153	validation_1-rmsle:0.06235	validation_2-rmsle:0.06250
[194]	validation_0-rmsle:0.06151	validation_1-rmsle:0.06233	validation_2-rmsle:0.06250
[195]	validation_0-rmsle:0.06150	validation_1-rmsle:0.06232	validation_2-rmsle:0.06249
[196]	validation_0-rmsle:0.06145	validation_1-rmsle:0.06228	validation_2-rmsle:0.06245
[197]	validation_0-rmsle:0.06144	validation_1-rmsle:0.06227	validation_2-rmsle:0.06244
[198]	validation_0-rmsle:0.06139	validation_1-rmsle:0.06224	validation_2-rmsle:0.06241
[199]	validation_0-rmsle:0.06134	validation_1-rmsle:0.06217	validation_2-rmsle:0.06236
[200]	validation_0-rmsle:0.06128	validation

[284]	validation_0-rmsle:0.05907	validation_1-rmsle:0.06044	validation_2-rmsle:0.06047
[285]	validation_0-rmsle:0.05906	validation_1-rmsle:0.06043	validation_2-rmsle:0.06046
[286]	validation_0-rmsle:0.05899	validation_1-rmsle:0.06035	validation_2-rmsle:0.06040
[287]	validation_0-rmsle:0.05897	validation_1-rmsle:0.06034	validation_2-rmsle:0.06038
[288]	validation_0-rmsle:0.05895	validation_1-rmsle:0.06032	validation_2-rmsle:0.06037
[289]	validation_0-rmsle:0.05893	validation_1-rmsle:0.06030	validation_2-rmsle:0.06033
[290]	validation_0-rmsle:0.05890	validation_1-rmsle:0.06028	validation_2-rmsle:0.06032
[291]	validation_0-rmsle:0.05888	validation_1-rmsle:0.06025	validation_2-rmsle:0.06031
[292]	validation_0-rmsle:0.05886	validation_1-rmsle:0.06023	validation_2-rmsle:0.06029
[293]	validation_0-rmsle:0.05885	validation_1-rmsle:0.06022	validation_2-rmsle:0.06028
[294]	validation_0-rmsle:0.05883	validation_1-rmsle:0.06022	validation_2-rmsle:0.06028
[295]	validation_0-rmsle:0.05882	validation

[379]	validation_0-rmsle:0.05723	validation_1-rmsle:0.05928	validation_2-rmsle:0.05928
[380]	validation_0-rmsle:0.05719	validation_1-rmsle:0.05926	validation_2-rmsle:0.05926
[381]	validation_0-rmsle:0.05717	validation_1-rmsle:0.05925	validation_2-rmsle:0.05925
[382]	validation_0-rmsle:0.05717	validation_1-rmsle:0.05925	validation_2-rmsle:0.05925
[383]	validation_0-rmsle:0.05716	validation_1-rmsle:0.05924	validation_2-rmsle:0.05925
[384]	validation_0-rmsle:0.05715	validation_1-rmsle:0.05923	validation_2-rmsle:0.05923
[385]	validation_0-rmsle:0.05713	validation_1-rmsle:0.05923	validation_2-rmsle:0.05922
[386]	validation_0-rmsle:0.05712	validation_1-rmsle:0.05924	validation_2-rmsle:0.05922
[387]	validation_0-rmsle:0.05710	validation_1-rmsle:0.05924	validation_2-rmsle:0.05921
[388]	validation_0-rmsle:0.05710	validation_1-rmsle:0.05923	validation_2-rmsle:0.05921
[389]	validation_0-rmsle:0.05710	validation_1-rmsle:0.05923	validation_2-rmsle:0.05921
[390]	validation_0-rmsle:0.05707	validation

...

[1137]	validation_0-rmsle:0.05052	validation_1-rmsle:0.05605	validation_2-rmsle:0.05601
[1138]	validation_0-rmsle:0.05052	validation_1-rmsle:0.05605	validation_2-rmsle:0.05601
[1139]	validation_0-rmsle:0.05051	validation_1-rmsle:0.05605	validation_2-rmsle:0.05601
[1140]	validation_0-rmsle:0.05051	validation_1-rmsle:0.05605	validation_2-rmsle:0.05600
[1141]	validation_0-rmsle:0.05050	validation_1-rmsle:0.05605	validation_2-rmsle:0.05600
[1142]	validation_0-rmsle:0.05049	validation_1-rmsle:0.05605	validation_2-rmsle:0.05599
[1143]	validation_0-rmsle:0.05048	validation_1-rmsle:0.05604	validation_2-rmsle:0.05599
[1144]	validation_0-rmsle:0.05047	validation_1-rmsle:0.05604	validation_2-rmsle:0.05599
[1145]	validation_0-rmsle:0.05046	validation_1-rmsle:0.05604	validation_2-rmsle:0.05599
[1146]	validation_0-rmsle:0.05046	validation_1-rmsle:0.05604	validation_2-rmsle:0.05598
[1147]	validation_0-rmsle:0.05045	validation_1-rmsle:0.05604	validation_2-rmsle:0.05598
[1148]	validation_0-rmsle:0.0504

[1231]	validation_0-rmsle:0.04998	validation_1-rmsle:0.05587	validation_2-rmsle:0.05582
[1232]	validation_0-rmsle:0.04998	validation_1-rmsle:0.05587	validation_2-rmsle:0.05582
[1233]	validation_0-rmsle:0.04997	validation_1-rmsle:0.05587	validation_2-rmsle:0.05581
[1234]	validation_0-rmsle:0.04997	validation_1-rmsle:0.05587	validation_2-rmsle:0.05581
[1235]	validation_0-rmsle:0.04996	validation_1-rmsle:0.05587	validation_2-rmsle:0.05581
[1236]	validation_0-rmsle:0.04995	validation_1-rmsle:0.05587	validation_2-rmsle:0.05581
[1237]	validation_0-rmsle:0.04995	validation_1-rmsle:0.05587	validation_2-rmsle:0.05581
[1238]	validation_0-rmsle:0.04994	validation_1-rmsle:0.05587	validation_2-rmsle:0.05581
[1239]	validation_0-rmsle:0.04993	validation_1-rmsle:0.05586	validation_2-rmsle:0.05579
[1240]	validation_0-rmsle:0.04993	validation_1-rmsle:0.05586	validation_2-rmsle:0.05580
[1241]	validation_0-rmsle:0.04991	validation_1-rmsle:0.05584	validation_2-rmsle:0.05579
[1242]	validation_0-rmsle:0.0499

[1325]	validation_0-rmsle:0.04937	validation_1-rmsle:0.05566	validation_2-rmsle:0.05557
[1326]	validation_0-rmsle:0.04937	validation_1-rmsle:0.05566	validation_2-rmsle:0.05557
[1327]	validation_0-rmsle:0.04937	validation_1-rmsle:0.05566	validation_2-rmsle:0.05557
[1328]	validation_0-rmsle:0.04936	validation_1-rmsle:0.05566	validation_2-rmsle:0.05557
[1329]	validation_0-rmsle:0.04935	validation_1-rmsle:0.05565	validation_2-rmsle:0.05556
[1330]	validation_0-rmsle:0.04934	validation_1-rmsle:0.05565	validation_2-rmsle:0.05556
[1331]	validation_0-rmsle:0.04933	validation_1-rmsle:0.05565	validation_2-rmsle:0.05555
[1332]	validation_0-rmsle:0.04933	validation_1-rmsle:0.05565	validation_2-rmsle:0.05556
[1333]	validation_0-rmsle:0.04932	validation_1-rmsle:0.05565	validation_2-rmsle:0.05556
[1334]	validation_0-rmsle:0.04931	validation_1-rmsle:0.05565	validation_2-rmsle:0.05556
[1335]	validation_0-rmsle:0.04931	validation_1-rmsle:0.05565	validation_2-rmsle:0.05555
[1336]	validation_0-rmsle:0.0493

[1419]	validation_0-rmsle:0.04895	validation_1-rmsle:0.05556	validation_2-rmsle:0.05547
[1420]	validation_0-rmsle:0.04895	validation_1-rmsle:0.05556	validation_2-rmsle:0.05547
[1421]	validation_0-rmsle:0.04895	validation_1-rmsle:0.05556	validation_2-rmsle:0.05547
[1422]	validation_0-rmsle:0.04895	validation_1-rmsle:0.05555	validation_2-rmsle:0.05547
[1423]	validation_0-rmsle:0.04894	validation_1-rmsle:0.05555	validation_2-rmsle:0.05547
[1424]	validation_0-rmsle:0.04894	validation_1-rmsle:0.05555	validation_2-rmsle:0.05546
[1425]	validation_0-rmsle:0.04894	validation_1-rmsle:0.05555	validation_2-rmsle:0.05546
[1426]	validation_0-rmsle:0.04893	validation_1-rmsle:0.05555	validation_2-rmsle:0.05546
[1427]	validation_0-rmsle:0.04892	validation_1-rmsle:0.05555	validation_2-rmsle:0.05546
[1428]	validation_0-rmsle:0.04892	validation_1-rmsle:0.05555	validation_2-rmsle:0.05546
[1429]	validation_0-rmsle:0.04891	validation_1-rmsle:0.05555	validation_2-rmsle:0.05546
[1430]	validation_0-rmsle:0.0489

[1513]	validation_0-rmsle:0.04856	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1514]	validation_0-rmsle:0.04855	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1515]	validation_0-rmsle:0.04855	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1516]	validation_0-rmsle:0.04855	validation_1-rmsle:0.05548	validation_2-rmsle:0.05538
[1517]	validation_0-rmsle:0.04855	validation_1-rmsle:0.05548	validation_2-rmsle:0.05538
[1518]	validation_0-rmsle:0.04855	validation_1-rmsle:0.05548	validation_2-rmsle:0.05538
[1519]	validation_0-rmsle:0.04854	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1520]	validation_0-rmsle:0.04854	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1521]	validation_0-rmsle:0.04853	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1522]	validation_0-rmsle:0.04853	validation_1-rmsle:0.05548	validation_2-rmsle:0.05538
[1523]	validation_0-rmsle:0.04853	validation_1-rmsle:0.05548	validation_2-rmsle:0.05537
[1524]	validation_0-rmsle:0.0485

In [69]:
from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

In [71]:

xgb = XGBRegressor(nthread=-1) 

param_dist = {'n_estimators': [3000],
              'learning_rate': stats.uniform(0.01, 0.4),
              'subsample': stats.uniform(0.3, 0.9),
              'gamma':stats.uniform(0, 1),
              'max_depth': [3,4,5, 6, 7,8,9],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': stats.uniform(0, 1),
              'subsample':stats.uniform(0.5, 1),
               'colsample_bytree':stats.uniform(0.5, 1),
             }
rgrid = RandomizedSearchCV(xgb, param_distributions = param_dist,scoring='neg_mean_squared_log_error',verbose=3,n_jobs=4,cv=3)
model  = rgrid.fit(x_train,y_train,verbose=True,eval_metric=['rmsle'],eval_set=eval_set,early_stopping_rounds=20)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:   47.1s finished


[0]	validation_0-rmsle:0.78660	validation_1-rmsle:0.78863	validation_2-rmsle:0.78462
Multiple eval metrics have been passed: 'validation_2-rmsle' will be used for early stopping.

Will train until validation_2-rmsle hasn't improved in 20 rounds.
[1]	validation_0-rmsle:0.54856	validation_1-rmsle:0.55085	validation_2-rmsle:0.54663
[2]	validation_0-rmsle:0.39750	validation_1-rmsle:0.39933	validation_2-rmsle:0.39551
[3]	validation_0-rmsle:0.29715	validation_1-rmsle:0.29863	validation_2-rmsle:0.29513
[4]	validation_0-rmsle:0.22731	validation_1-rmsle:0.22868	validation_2-rmsle:0.22556
[5]	validation_0-rmsle:0.17743	validation_1-rmsle:0.17855	validation_2-rmsle:0.17610
[6]	validation_0-rmsle:0.14259	validation_1-rmsle:0.14366	validation_2-rmsle:0.14176
[7]	validation_0-rmsle:0.11786	validation_1-rmsle:0.11895	validation_2-rmsle:0.11712
[8]	validation_0-rmsle:0.10161	validation_1-rmsle:0.10254	validation_2-rmsle:0.10100
[9]	validation_0-rmsle:0.09038	validation_1-rmsle:0.09123	validation_2-rms

[94]	validation_0-rmsle:0.05849	validation_1-rmsle:0.06069	validation_2-rmsle:0.06026
[95]	validation_0-rmsle:0.05843	validation_1-rmsle:0.06062	validation_2-rmsle:0.06017
[96]	validation_0-rmsle:0.05837	validation_1-rmsle:0.06060	validation_2-rmsle:0.06014
[97]	validation_0-rmsle:0.05833	validation_1-rmsle:0.06054	validation_2-rmsle:0.06011
[98]	validation_0-rmsle:0.05830	validation_1-rmsle:0.06051	validation_2-rmsle:0.06011
[99]	validation_0-rmsle:0.05816	validation_1-rmsle:0.06045	validation_2-rmsle:0.06002
[100]	validation_0-rmsle:0.05809	validation_1-rmsle:0.06040	validation_2-rmsle:0.05995
[101]	validation_0-rmsle:0.05808	validation_1-rmsle:0.06038	validation_2-rmsle:0.05996
[102]	validation_0-rmsle:0.05796	validation_1-rmsle:0.06034	validation_2-rmsle:0.05989
[103]	validation_0-rmsle:0.05784	validation_1-rmsle:0.06030	validation_2-rmsle:0.05982
[104]	validation_0-rmsle:0.05781	validation_1-rmsle:0.06029	validation_2-rmsle:0.05977
[105]	validation_0-rmsle:0.05775	validation_1-rms

[189]	validation_0-rmsle:0.05410	validation_1-rmsle:0.05824	validation_2-rmsle:0.05752
[190]	validation_0-rmsle:0.05404	validation_1-rmsle:0.05820	validation_2-rmsle:0.05751
[191]	validation_0-rmsle:0.05398	validation_1-rmsle:0.05819	validation_2-rmsle:0.05751
[192]	validation_0-rmsle:0.05397	validation_1-rmsle:0.05817	validation_2-rmsle:0.05750
[193]	validation_0-rmsle:0.05392	validation_1-rmsle:0.05808	validation_2-rmsle:0.05746
[194]	validation_0-rmsle:0.05388	validation_1-rmsle:0.05808	validation_2-rmsle:0.05747
[195]	validation_0-rmsle:0.05385	validation_1-rmsle:0.05807	validation_2-rmsle:0.05745
[196]	validation_0-rmsle:0.05382	validation_1-rmsle:0.05806	validation_2-rmsle:0.05745
[197]	validation_0-rmsle:0.05377	validation_1-rmsle:0.05804	validation_2-rmsle:0.05744
[198]	validation_0-rmsle:0.05373	validation_1-rmsle:0.05799	validation_2-rmsle:0.05739
[199]	validation_0-rmsle:0.05372	validation_1-rmsle:0.05799	validation_2-rmsle:0.05739
[200]	validation_0-rmsle:0.05368	validation

[284]	validation_0-rmsle:0.05114	validation_1-rmsle:0.05700	validation_2-rmsle:0.05630
[285]	validation_0-rmsle:0.05112	validation_1-rmsle:0.05701	validation_2-rmsle:0.05634
[286]	validation_0-rmsle:0.05107	validation_1-rmsle:0.05704	validation_2-rmsle:0.05632
[287]	validation_0-rmsle:0.05104	validation_1-rmsle:0.05704	validation_2-rmsle:0.05632
[288]	validation_0-rmsle:0.05101	validation_1-rmsle:0.05706	validation_2-rmsle:0.05629
[289]	validation_0-rmsle:0.05098	validation_1-rmsle:0.05703	validation_2-rmsle:0.05625
[290]	validation_0-rmsle:0.05095	validation_1-rmsle:0.05702	validation_2-rmsle:0.05625
[291]	validation_0-rmsle:0.05093	validation_1-rmsle:0.05700	validation_2-rmsle:0.05624
[292]	validation_0-rmsle:0.05091	validation_1-rmsle:0.05702	validation_2-rmsle:0.05623
[293]	validation_0-rmsle:0.05089	validation_1-rmsle:0.05706	validation_2-rmsle:0.05620
[294]	validation_0-rmsle:0.05088	validation_1-rmsle:0.05706	validation_2-rmsle:0.05620
[295]	validation_0-rmsle:0.05087	validation

Stopping. Best iteration:
[358]	validation_0-rmsle:0.04949	validation_1-rmsle:0.05650	validation_2-rmsle:0.05600



In [72]:
np.sqrt(mean_squared_log_error(y_valid,model.predict(x_valid)))

0.0560030475271017

In [73]:
model.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6459591424767369,
             gamma=0.14627227408978427, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.22766592400502914,
             max_delta_step=0, max_depth=4, min_child_weight=0.7629283031302394,
             missing=nan, monotone_constraints='()', n_estimators=3000,
             n_jobs=-1, nthread=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.6155820886940446, tree_method='exact',
             validate_parameters=1, verbosity=None)